# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,25.29,73,9,3.87,PN,1737842362
1,1,cockburn town,21.4612,-71.1419,25.98,100,0,5.66,TC,1737842363
2,2,ushuaia,-54.8000,-68.3000,14.81,44,20,7.20,AR,1737842365
3,3,deniliquin,-35.5333,144.9667,22.49,42,80,3.84,AU,1737842366
4,4,new norfolk,-42.7826,147.0587,14.28,55,76,1.07,AU,1737842367


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    frame_width=800,
    frame_height=600,
    color="City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# I filtered for Max Temp between 20 and 30 C, Humidity between 30 and 70, and Cloudiness less that 50%.
ideal_weather_df = city_data_df[(city_data_df["Max Temp"]>=20) &
             (city_data_df['Max Temp']<=30) &
             (city_data_df["Humidity"] >=30) &
             (city_data_df["Humidity"]<=70) &
             (city_data_df["Cloudiness"]<50)]

# Drop any rows with null values
ideal_weather_df.info()  #to check for null values
# N/A all the filtered columns contained data. If there were missing values I would use the following code:
#clean_ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 15 to 531
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     28 non-null     int64  
 1   City        28 non-null     object 
 2   Lat         28 non-null     float64
 3   Lng         28 non-null     float64
 4   Max Temp    28 non-null     float64
 5   Humidity    28 non-null     int64  
 6   Cloudiness  28 non-null     int64  
 7   Wind Speed  28 non-null     float64
 8   Country     28 non-null     object 
 9   Date        28 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 2.4+ KB


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,brisas de zicatela,15.8369,-97.0419,27.91,52,1,2.14,MX,1737842380
31,31,caraballeda,10.6112,-66.8522,27.04,65,4,5.14,VE,1737842399
39,39,arraial do cabo,-22.9661,-42.0278,27.98,65,40,7.20,BR,1737842410
40,40,fale old settlement,-9.3852,-171.2468,28.60,65,5,7.80,TK,1737842412
110,110,monte patria,-30.6919,-70.9467,28.12,31,0,3.34,CL,1737842496
115,115,sweetwater,25.7634,-80.3731,21.12,63,40,7.20,US,1737842504
140,140,pak chong,14.7080,101.4161,21.01,46,3,1.05,TH,1737842536
195,195,bloemhof,-27.6469,25.6070,24.32,34,0,0.97,ZA,1737842604
196,196,callao,-12.0667,-77.1500,25.16,65,0,6.17,PE,1737842605
200,200,acapulco de juarez,16.8634,-99.8901,25.86,67,2,3.90,MX,1737842502


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country","Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,brisas de zicatela,MX,15.8369,-97.0419,52,
31,caraballeda,VE,10.6112,-66.8522,65,
39,arraial do cabo,BR,-22.9661,-42.0278,65,
40,fale old settlement,TK,-9.3852,-171.2468,65,
110,monte patria,CL,-30.6919,-70.9467,31,
115,sweetwater,US,25.7634,-80.3731,63,
140,pak chong,TH,14.7080,101.4161,46,
195,bloemhof,ZA,-27.6469,25.6070,34,
196,callao,PE,-12.0667,-77.1500,65,
200,acapulco de juarez,MX,16.8634,-99.8901,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"}    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
   
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
brisas de zicatela - nearest hotel: Casa de Olas
caraballeda - nearest hotel: Hotel Belmar
arraial do cabo - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
monte patria - nearest hotel: Hotel Monte Cordillera
sweetwater - nearest hotel: Courtyard Miami at Dolphin Mall
pak chong - nearest hotel: Phubade hotel
bloemhof - nearest hotel: Calimyn
callao - nearest hotel: Casa Ronald
acapulco de juarez - nearest hotel: Hotel del Valle
san luis de la loma - nearest hotel: No hotel found
bikenibeu village - nearest hotel: Otintaai Hotel
la cruz - nearest hotel: Hotel Las Palmas
maracai - nearest hotel: No hotel found
chui - nearest hotel: Alerces
salalah - nearest hotel: Muscat International Hotel
kailua-kona - nearest hotel: PACIFIC 19 Kona
kitgum - nearest hotel: Cactus Hillside Hotel
boende - nearest hotel: Hotel Reference
castillos - nearest hotel: A mi gente
holualoa - nearest hotel: Kona Hotel
cabo san lucas - nearest hotel: Comfort

,City,Country,Lat,Lng,Humidity,Hotel Name
15,brisas de zicatela,MX,15.8369,-97.0419,52,Casa de Olas
31,caraballeda,VE,10.6112,-66.8522,65,Hotel Belmar
39,arraial do cabo,BR,-22.9661,-42.0278,65,No hotel found
40,fale old settlement,TK,-9.3852,-171.2468,65,No hotel found
110,monte patria,CL,-30.6919,-70.9467,31,Hotel Monte Cordillera
115,sweetwater,US,25.7634,-80.3731,63,Courtyard Miami at Dolphin Mall
140,pak chong,TH,14.7080,101.4161,46,Phubade hotel
195,bloemhof,ZA,-27.6469,25.6070,34,Calimyn
196,callao,PE,-12.0667,-77.1500,65,Casa Ronald
200,acapulco de juarez,MX,16.8634,-99.8901,67,Hotel del Valle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    frame_width = 800,
    frame_height = 600,
    size=50,
    hover_cols=["Country", "Hotel Name", "Humidity"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)